In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pydeck as pdk

# 포스트 데이터 불러오기
df_post = pd.read_csv('../inputs/posts/garosoogil.csv', index_col=0)

# 텍스트 없으면 drop
df_post = df_post.dropna()

# 12월로 필터링
df_post = df_post.loc[df_post.loc[:,'date'].str.contains('2023-12')]

In [9]:
# 상점 데이터 불러오기
gdf_store_garosoogil = gpd.read_file('../inputs/stores/store_garosoogil.geojson')

# 음식점만 필터링
gdf_store_garosoogil = gdf_store_garosoogil.loc[gdf_store_garosoogil.loc[:,'상권업종대분류명'] == '음식']

# 상호명 없는 상점은 삭제
gdf_store_garosoogil = gdf_store_garosoogil.dropna(subset='상호명')

# 상호명과 geometry만 남기고 다 삭제 (현재 필요없는 컬럼들)
gdf_store_garosoogil = gdf_store_garosoogil.loc[:,['상호명', 'geometry']]

# 상호명: store_name으로 변경
gdf_store_garosoogil = gdf_store_garosoogil.rename(columns={'상호명':'store_name'})

In [10]:
gdf_store_garosoogil.shape

(512, 2)

In [11]:
def counting_post(store_name):
    '''
    상점 이름을 input으로 받아 posting 갯수를 세주는 함수

    input:
        store_name (string): 상호명
    output:
        posting_count (int): 포스팅 갯수

    '''

    # 상호명이 한글자이면 그냥 0개로 취급
    if len(store_name) == 1:
        return 0

    # 포스트 중에서 상점 이름이 포함되어 있으면 좌표 추가
    posting_count = df_post.loc[df_post.loc[:,'text'].str.contains(store_name, regex=False)].shape[0]

    return posting_count


In [12]:
gdf_store_garosoogil.loc[:,'posting_count'] = gdf_store_garosoogil.loc[:,'store_name'].apply(counting_post)

In [13]:
# 포스팅 갯수 기준으로 정렬
gdf_store_garosoogil = gdf_store_garosoogil.sort_values(by='posting_count', ascending=False)

In [14]:
for i, row in gdf_store_garosoogil.iterrows():
    print(row['store_name'], row['posting_count'])
    if row['posting_count'] == 0:
        break

사이 88
프레임 67
신사동술집 64
포인트 60
센스 12
해가 9
브리온 8
컬렉션 8
우와 8
우연 6
페타테 6
러쉬 6
마사 5
옥토 5
파치노 4
진도켄 4
소각 4
우니도 4
자매의부엌 4
버드맨 4
파치노 4
어덜트온리 3
소나 3
유희 3
신사전 3
멘츠루 3
핫쵸 3
샤블리 2
감성타코 2
육구공 2
소양탐정 2
사이카이 2
돈불리제담 2
고우 2
미미면가 2
공감 2
플롭 2
타코앤칠리 2
채운 2
금별맥주 2
장흥집 2
해남집 2
동녘 2
낭낙 2
미미면가 2
갓포우오 2
루카 2
천미미 2
온달집 1
서리달 1
우오하나 1
켄카 1
안다오 1
가로수그곳 1
어썸로즈 1
프레즈드로이 1
해밥상달주막 1
올데이사운드바 1
원조마포소금구이 1
불공장 1
카페구오 1
온다플레이버 1
밀떡팩토리 1
연경 1
야사이마끼쿠이신보 1
대봉집 1
바다를훔친언니 1
히츠지야가로수길점 1
야키토리요이사 1
라키 1
온기정 1
포사로 1
더블트러블 1
신사참숯불고기 1
테일러커피 1
팔일삼 1
컬렉션서울 1
강남주막다방 1
양식당더램키친 1
듀자미 1
삼바리 1
송옥 1
네기 1
한올 1
신사숙녀 1
육회담 1
희래등 1
카페포엠 1
아로보서울 1
뮤즈온 1
보바베어 1
에이블 1
미분당 1
삼창교자 1
미미곱창 1
파니엔테 1
이치서울 1
신사소곱창 0


In [19]:
list_stopwords = [
    '압구정',
'만나',
'오픈',
'라인',
'라인',
'사이',
'사이',
'리뷰',
'랜드',
'프레임',
'신사동술집',
'일리',
'이나',
'부부',
'포인트',
'데일리',
'화이트',
'베이',
'와인사랑',
'제이',
'크림',
'크림',
'크림',
'지나',
'하우',
'카이',
'카이',
'자이',
'압구정술집',
'지니',
'이즈',
'그레이',
'소스',
'아가',
'컴퍼니',
'힐링',
'강남사',
'센스',
'우와'
]

In [20]:
# list_stopwords 안에 있는 상점들은 우선 데이터프레임에서 제외
gdf_store_garosoogil = gdf_store_garosoogil.loc[~gdf_store_garosoogil.loc[:,'store_name'].isin(list_stopwords)]

# 포스팅 수가 0인 것들도 데이터프레임에서 제외
gdf_store_garosoogil = gdf_store_garosoogil.loc[gdf_store_garosoogil.loc[:,'posting_count']>0]

In [21]:
gdf_store_garosoogil

,store_name,geometry,posting_count
1553,해가,POINT (127.02132 37.51849),9
1637,브리온,POINT (127.02058 37.52288),8
756,컬렉션,POINT (127.02218 37.52145),8
176,우연,POINT (127.02424 37.51868),6
1241,페타테,POINT (127.02328 37.52225),6
...,...,...,...
620,미분당,POINT (127.02006 37.51785),1
154,삼창교자,POINT (127.02173 37.51891),1
1777,미미곱창,POINT (127.02129 37.51875),1
1768,파니엔테,POINT (127.02215 37.51884),1


In [22]:
gdf_store_garosoogil.loc[:,'posting_count'].sum()

190

In [23]:
# pydeck layer 설정

scatterplot_layer = pdk.Layer(
    'ScatterplotLayer',
    gdf_store_garosoogil,
    pickable=True,
    opacity=0.3,
    stroked=False,
    filled=True,
    radius_scale=2,
    get_position='geometry.coordinates',
    get_radius='posting_count',
    get_fill_color=[255,140,0]
)

In [25]:
# Set the viewport location
view_state = pdk.ViewState(latitude=37.5210, longitude=127.0228, zoom=15, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[scatterplot_layer], initial_view_state=view_state, tooltip={"text": "{store_name}\n{posting_count}"})
r.to_html("../output/maps/garosoogil/scatterplot_layer.html")

In [26]:
heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=gdf_store_garosoogil,
    opacity=0.5,
    get_position='geometry.coordinates',
    aggregation=pdk.types.String("MEAN"),
    get_weight="posting_count",
)

In [27]:
r = pdk.Deck(
    layers=[heatmap_layer],
    initial_view_state=view_state
)

r.to_html("../output/maps/garosoogil/heatmap_layer.html")